## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/Weather_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hvide Sande,56.0045,8.1294,40.08,71,51,13.13,DK,2022-04-28 00:20:09
1,1,New Norfolk,-42.7826,147.0587,63.91,86,94,1.99,AU,2022-04-28 00:20:32
2,2,Hobart,-42.8794,147.3294,64.26,89,75,6.91,AU,2022-04-28 00:15:03
3,3,New Norfolk,-42.7826,147.0587,63.91,86,94,1.99,AU,2022-04-28 00:20:32
4,4,Hobart,-42.8794,147.3294,64.26,89,75,6.91,AU,2022-04-28 00:15:03


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.count()

Unnamed: 0    137
City          137
Lat           137
Lng           137
Max Temp      137
Humidity      137
Cloudiness    137
Wind Speed    137
Country       136
Date          137
dtype: int64

In [5]:
# 4a. Determine if there are any empty rows.
pd.isnull(preferred_cities_df)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,False,False,False,False,False,False,False,False,False,False
13,False,False,False,False,False,False,False,False,False,False
35,False,False,False,False,False,False,False,False,False,False
41,False,False,False,False,False,False,False,False,False,False
52,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
648,False,False,False,False,False,False,False,False,False,False
653,False,False,False,False,False,False,False,False,False,False
655,False,False,False,False,False,False,False,False,False,False
659,False,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
mod_cities = preferred_cities_df.dropna()
mod_cities.count()

Unnamed: 0    136
City          136
Lat           136
Lng           136
Max Temp      136
Humidity      136
Cloudiness    136
Wind Speed    136
Country       136
Date          136
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = mod_cities[["City", "Country", "Max Temp", "Cloudiness", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Cloudiness,Lat,Lng,Hotel Name
9,Bengkulu,ID,76.60,86,-3.8004,102.2655,
13,Rikitea,PF,77.27,13,-23.1203,-134.9692,
35,Puerto Ayora,EC,72.84,14,-0.7393,-90.3518,
41,Saint-Georges,GD,78.48,40,12.0564,-61.7485,
52,Paita,PE,70.32,11,-5.0892,-81.1144,
57,Touros,BR,78.98,98,-5.1989,-35.4608,
66,Atuona,PF,79.07,20,-9.8000,-139.0333,
81,Gympie,AU,72.18,83,-26.1898,152.6650,
82,Naze,JP,72.52,99,28.3667,129.4833,
83,Port Lincoln,AU,72.48,95,-34.7333,135.8667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Create list of lat and lng to pass to api
    params["location"] = f"{lat},{lng}"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.")
    
print('API call complete')
        

Hotel not found... skipping.
API call complete


In [9]:
#hotel_df.count()
#hotel_df.isnull()
hotel_df.tail()

,City,Country,Max Temp,Cloudiness,Lat,Lng,Hotel Name
648,Gouyave,GD,77.77,40,12.1646,-61.7296,Rumboat Retreat
653,Angoche,MZ,76.51,98,-16.2325,39.9086,Guest House HESADA
655,Le Moule,GP,79.18,93,16.3332,-61.3473,Lakaz Hananou
659,Muisne,EC,77.77,97,0.6000,-80.0333,Quinta Samay
661,Sao Bento,BR,71.58,100,-6.2833,-37.7500,


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City          136
Country       136
Max Temp      136
Cloudiness    136
Lat           136
Lng           136
Hotel Name    136
dtype: int64

In [11]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('WeatherPy_vacation.csv', index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Cloudiness %</dt><dd>{Cloudiness}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
locations.info()

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))